In [ ]:
import numpy as np

In [ ]:
class node():
    def __init__(self, ind=None, name=None, ziyuan=None, assignment=None, net=None):
        self.id = ind
        self.name = name
        self.ziyuan = ziyuan         # 节点所用资源的列表(在当前题目要求下，资源总和即可，不需要列表)
        self.assignment = assignment # 节点被分配到的FPGA序号
        self.net = net               # 节点所属线网的序号


class net():
    def __init__(self, nodelist=None):
        self.nodelist = nodelist        # 线网中的节点的列表，此列表中每个元素是节点的名字
        self.assignment = set()    # 线网中的节点被分配到的FPGA的集合， 此集合中每个元素是FPGA的序号


In [ ]:
def readNodeFile(filename):
    f = open(filename)
    vertex = {}        # 字典
    vertexname=[]      # 列表
    ind = 0
    for line in f:
        list1 = line.split()
        nm = list1[0]
        vertexname.append(nm)
        ziyuan = list(map(int, list1[-10:]))
        vertex[nm] = node(ind, nm, sum(ziyuan))
        ind += 1
    f.close()
    return vertex,vertexname

In [ ]:
def readNetFile(filename,vertex):
    f = open(filename)
    linestack=[]
    for line in f:
        linestack.append(line.split())
    nets=[]
    nodelist=[]
    ind=0
    while linestack!=[]:
        line = linestack.pop()
        nodelist.append(line[0])
        if 's' in line:
            nets.append(net(nodelist))
            for node in nodelist:
                vertex[node].net = ind
            nodelist=[]
            ind += 1
    f.close()
    return nets

In [ ]:
def fenpeiMethod(vertex, nosFPGA,vertexname):
    return randomFenpei(vertex, nosFPGA,vertexname)

def randomFenpei(vertex, nosFPGA,vertexname):
    nosnode = len(vertex)
    fenpei=[]
    for i in range(nosnode):
        fenpei.append(np.random.randint(nosFPGA))
    F = [[] for i in range(nosFPGA)]
    for i in range(nosnode):
        F[fenpei[i]].append(i)
        vertex[vertexname[i]].assignment = fenpei[i]
        for onenet in nets:
            if vertexname[i] in onenet.nodelist:
                onenet.assignment.add(fenpei[i])    
    return F

In [ ]:
def writeResult(F,vertexname):
    nosFPGA = len(F)
    for fpga in range(nosFPGA):
        f.write("F"+str(fpga))
        for onenode in F[fpga]:
            f.write(" "+vertexname[onenode])
        f.write("\n")

In [ ]:
def ziyuanCal(F):
    nosFPGA = len(F)
    Z = []
    for i in range(nosFPGA):
        list1 = F[i]
        Z.append(sum(list1))
    return Z

In [ ]:
def linkCal(nets,nosFPGA):
    L = [0 for i in range(nosFPGA)]
    for onenet in nets:
        if len(onenet.assignment)==1:
            continue
        for fpgaInd in onenet.assignment:
            L[fpgaInd] += 1
    return L


In [ ]:
def clearAssignInfo(vertex,vertexname,nets):
    for nodeInd in range(len(vertexname)):
        vertex[vertexname[nodeInd]].assignment = None
    for onenet in nets:
        onenet.assignment = set()

In [ ]:
# 读取节点文件
filename = "./contest/testdata-0/design.are"
vertex,vertexname = readNodeFile(filename)

In [ ]:
# 读取线网文件
filename = "./contest/testdata-0/design.net"
nets = readNetFile(filename,vertex)

In [ ]:
print(nets[3].nodelist)
print(nets[4].assignment)

In [ ]:
nosFPGA = 4

In [ ]:
filename = "./result.res"
f = open(filename,'w')
np.random.seed(np.random.randint(100))

In [ ]:
# 第一种情况
clearAssignInfo(vertex,vertexname,nets)
F = fenpeiMethod(vertex,nosFPGA,vertexname)
writeResult(F,vertexname)
f.write(str(np.std(ziyuanCal(F)))+"\n")

In [ ]:
# 第二种情况
clearAssignInfo(vertex,vertexname,nets)
F = fenpeiMethod(vertex,nosFPGA,vertexname)
writeResult(F,vertexname)
f.write(str(sum(linkCal(nets,nosFPGA)))+"\n")

In [ ]:
# 第三种情况
clearAssignInfo(vertex,vertexname,nets)
F = fenpeiMethod(vertex,nosFPGA,vertexname)
writeResult(F,vertexname)
f.write(str(sum(linkCal(nets,nosFPGA)))+"\n")

In [ ]:
f.close()